# 1. predicting adsorption for each adsorption data point using Vt&BET only

In [5]:
import pandas as pd
import numpy as np
import os 
BET_path = 'C:/Kai_Zhang/MachineLearning/Unified gas Adsorption/Treated/CO2'

def pred_dataset(file_names):
    source_path = 'C:/Kai_Zhang/MachineLearning/Unified gas Adsorption/CO2_adsorption/new_data'
    train_df = pd.DataFrame()
    for file_name in file_names:
        temp_data = pd.read_excel(os.path.join(source_path,file_name+'-04-19-2022.xlsx'),skiprows= 1 )
        #temp_data = temp_data.dropna(axis=0,how = 'any',subset = feature_set)
        train_df = pd.concat([train_df,temp_data],axis=0)
       
    return train_df

In [6]:
from scipy import interpolate
import numpy as np
import pandas as pd

def inter_data(x_old,y_old):
    
    f = interpolate.interp1d(x_old, y_old,'slinear') 

    """seg1 = np.linspace(min(x_old),min(x_old)+0.2,num=20)
    seg2 = np.linspace(min(x_old)+0.2,max(x_old)-0.2,num=12)
    seg3 = np.linspace(max(x_old)-0.2, max(x_old),num=20)
    x_new = np.append(seg1,seg2[1:-1])
    x_new = np.append(x_new,seg3)   """
    x_new = np.linspace(0.01, 0.99,num=35)
       
    """
        for ele in x_old:
            t = bi_find(x_new,ele)

            x_new[t] = ele
        
        if (len(x_new)>150):
            x_new = x_new[len(x_new)-150:]
         = np.arange(min(x_old), max(x_old), (max(x_old)-min(x_old))/150)
    """
    y_new = f(x_new)
    #X_feature = np.append(x_new.reshape(-1,1),y_new.reshape(-1,1),axis=1).reshape(2,35).tolist()
    #x_mat = np.append(x_mat,X_feature,axis=1)
       
    return x_new, y_new

In [7]:
data = pred_dataset(["CO2"])
data

,Unnamed: 0,Literature,Lines,GAC,GAS,E,S,A,B,V,...,%N,Temp(K),Pressure,Adsorp(mmol/g),Index,logP,logQ,logD,Label,Vmic_index
0,0,1.0,0,UC-600-2,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,1.98,298.0,0.048715,0.547018,0,-3.021763,-0.603274,2.418489,CO2,0
1,1,1.0,1,UC-600-2,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,1.98,298.0,0.104147,0.935902,0,-2.261948,-0.066244,2.195704,CO2,0
2,2,1.0,2,UC-600-2,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,1.98,298.0,0.137430,1.095365,0,-1.984640,0.091087,2.075727,CO2,0
3,3,1.0,3,UC-600-2,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,1.98,298.0,0.208412,1.502384,0,-1.568239,0.407053,1.975292,CO2,0
4,4,1.0,4,UC-600-2,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,1.98,298.0,0.266122,1.715469,0,-1.323800,0.539687,1.863487,CO2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23336,23431,NaN,35,NaN,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,NaN,298.0,2.962963,4.588613,2190,1.086190,1.523578,0.437388,CO2,0
23337,23432,NaN,36,NaN,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,NaN,298.0,4.027778,5.245077,2190,1.393215,1.657290,0.264075,CO2,0
23338,23433,NaN,37,NaN,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,NaN,298.0,5.972222,6.029895,2190,1.787119,1.796730,0.009610,CO2,0
23339,23434,NaN,38,NaN,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,NaN,298.0,7.939815,6.638302,2190,2.071890,1.892856,-0.179034,CO2,0


In [8]:
BET_df = pd.DataFrame()
cols = ["E","S","A","B","V","BET","Vt","Vmic","Vmeso","Temp(K)","Pressure","Adsorp(mmol/g)","Index"]
def bet_extraction(temp_df:pd.DataFrame):
    global BET_df
    literature = temp_df["Literature"].values[0]
    gac_name = temp_df["GAC"].values[0]
    print(literature,gac_name)
    try:
        if literature!=np.nan:
                bet_data = BET_path+'/'+str(literature)+"/"+gac_name+".csv"
                #print(bet_data)
                bet_curve = pd.read_csv(bet_data,header=None)
                x = bet_curve.iloc[:,0].values.tolist()
                y = bet_curve.iloc[:,1].values.tolist()
                _,new_y = inter_data(x,y)
                rows = len(temp_df)
                bet_matrix = [new_y for _ in range(rows)]
                bet_df = pd.DataFrame(bet_matrix,columns = ["BET_"+str(i) for i in range(len(new_y))])
                bet_df = pd.concat([bet_df,temp_df[cols].reset_index()],axis= 1,ignore_index=True)
                print(bet_df.shape)
                BET_df = pd.concat([BET_df,bet_df,],axis= 0,ignore_index=True)    

        #else:
        #    print("There is not leterature for this gac: {}".format(gac_name))         
    except:
        print("The BET curve for gac: {} from paper {} is not available.".format(gac_name,literature))
total_index = list(set(data["Index"].values.tolist()))
for index in total_index:
    temp_df =  data[data["Index"]==index]
    bet_extraction(temp_df)

1.0 UC-600-2
The BET curve for gac: UC-600-2 from paper 1.0 is not available.
1.0 UC-600-3
The BET curve for gac: UC-600-3 from paper 1.0 is not available.
1.0 UC-600-4
The BET curve for gac: UC-600-4 from paper 1.0 is not available.
1.0 UC-600-2
The BET curve for gac: UC-600-2 from paper 1.0 is not available.
1.0 UC-600-3
The BET curve for gac: UC-600-3 from paper 1.0 is not available.
1.0 UC-600-4
The BET curve for gac: UC-600-4 from paper 1.0 is not available.
1.0 UC-650-2
The BET curve for gac: UC-650-2 from paper 1.0 is not available.
1.0 UC-650-3
The BET curve for gac: UC-650-3 from paper 1.0 is not available.
1.0 UC-650-4
The BET curve for gac: UC-650-4 from paper 1.0 is not available.
1.0 UC-650-2
The BET curve for gac: UC-650-2 from paper 1.0 is not available.
1.0 UC-650-3
The BET curve for gac: UC-650-3 from paper 1.0 is not available.
1.0 UC-650-4
The BET curve for gac: UC-650-4 from paper 1.0 is not available.
1.0 UC-700-2
The BET curve for gac: UC-700-2 from paper 1.0 is n

# Using evenly selected BET cureve to find the feature importance of each region in the BET curve by the interval of 0.1

In [9]:
# data splitting
#from random import shuffle
from sklearn.model_selection import train_test_split
from numpy import random
def group_split(BET_df:pd.DataFrame,random_state = 42):
    indexs = list(set(BET_df["Index"].values.tolist()))
    #shuffle(indexs)
    _,test_index = train_test_split(indexs,test_size= 0.2,random_state =random_state)

    #test_index = random.choice(indexs,len(indexs)//5,replace=False).tolist()
    train_df = BET_df[~BET_df["Index"].isin(test_index)]
    test_df = BET_df[BET_df["Index"].isin(test_index)]
    return train_df,test_df

In [10]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor,\
    BaggingRegressor,ExtraTreesRegressor,RandomForestRegressor
from lightgbm import LGBMRegressor  
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
  
n_estimators = [50,100,120,150,180,200,300]

# define different models#,
models = [
    #('SVR',SVR(max_iter=100000)),
    #('DT',DecisionTreeRegressor(random_state=42)),\
    # ('ADBR',AdaBoostRegressor(random_state=42)), 
    #("GBR",GradientBoostingRegressor(random_state=42)),\
    #('BG',BaggingRegressor(random_state=42,n_jobs=-1)),
    #('ETR',ExtraTreesRegressor(random_state=42,n_jobs=-1)),\
    #('RF',RandomForestRegressor(n_jobs=-1,random_state=42)),
    ('LGBM',LGBMRegressor(n_jobs = -1,random_state = 42)),\
    #('MLPR',MLPRegressor(random_state=42))
    #('BGLGBM',BaggingRegressor(LGBMRegressor(n_estimators = 200, n_jobs = -1,random_state = 42), random_state=42,n_jobs=-1)),\
    #('BGETR',BaggingRegressor(ExtraTreesRegressor(n_estimators = 180,random_state=42,n_jobs=6),random_state=42,n_jobs=-1))
    ]

# set search parameters grid for different models
para_grids = { 
    'SVR':{'kernel':['linear','poly','rbf','sigmoid','precomputed']},\
    'DT':{'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson']},\
    'ADBR':{'n_estimators':n_estimators,'learning_rate':[0.1,0.5,1,2],'loss':['linear','square','exponential']},\
    'GBR':{'n_estimators':n_estimators,'learning_rate':[0.1,0.5,1,2]},\
    'BG':{'n_estimators':[10,50,100]},\
    'ETR':{'n_estimators':n_estimators},\
    'RF':{'n_estimators':n_estimators},\
    'LGBM':{'num_leaves':[10,20,30,50,],'learning_rate': [0.05,0.1,0.5,1],'n_estimators':n_estimators},\
    'BGLGBM':{'n_estimators':[10,30,50]},\
    'BGETR':{'n_estimators':[10]},\
    'MLPR':{'hidden_layer_sizes':[(4,100),(5,100),(6,100)],
            "activation":["logistic","relu"],\
            "solver":['sgd', 'adam']}
    }

In [11]:
from sklearn.model_selection import GridSearchCV,cross_validate,GroupKFold
from sklearn.ensemble import ExtraTreesRegressor
from  sklearn.metrics import mean_squared_error,r2_score
from sklearn.utils import shuffle

def model_CV(train_x,train_y,groups,model,para_grid):

    out_cv = GroupKFold(n_splits = 5)
    result = GridSearchCV(model,para_grid,cv= out_cv.get_n_splits(groups =groups),
    scoring='neg_mean_squared_error', return_train_score=True,n_jobs=-1)
    result.fit(train_x,train_y)
    
    model_refit =model.set_params(**result.best_params_)
    train_cv = cross_validate(model_refit,train_x,train_y,groups = groups,cv =out_cv,scoring = ('r2', 'neg_mean_squared_error'))
    train_mse_cv = -train_cv['test_neg_mean_squared_error'].mean()
    train_r2_cv = train_cv['test_r2'].mean()
    
    return [train_r2_cv,train_mse_cv],result.best_params_

# model evaluation
def model_eval(model,test_x,test_y):
      
    test_pre = model.predict(test_x)
    test_r2 = r2_score(test_pre,test_y)
    test_mse = mean_squared_error(test_y,test_pre)
    return test_r2,test_mse

# comparing different models
def model_comparison(model_list,para_grids,feature_list,gas_list):
    gas_list = gas_list 
    input_feature = feature_list
    output = ['Adsorp(mmol/g)']
    result_total = []
    for gas in gas_list:
        if gas =='total':
            train_df_com = train_df
            test_df_com = test_df
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            groups = train_df_com['Index'].values
            train_x, train_y, groups = shuffle(train_x, train_y, groups, random_state=42)
            
            for model_name, model in model_list:
                result, best_param = model_CV(train_x,train_y.squeeze(),groups,model,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                test_r2_total,test_mse_total = model_eval(model_refit,test_x,test_y.squeeze()) 
                for gases in gas_list[1:]:
                    test_df_com = test_df[test_df['Label']==gases]
                    test_xs = test_df_com[input_feature]
                    test_ys = test_df_com[output].values
                    test_r2,test_mse = model_eval(model_refit,test_xs,test_ys.squeeze()) 
                    result_total.append([gases,model_name+'_total',result[0],result[1],test_r2_total,test_mse_total,test_r2,test_mse,best_param])
                    print('Dataset {}, Algorithm {}, Test_r2 {}, Test_error {}'.format(gas,model_name+'_total',test_r2,test_mse))    
        else:
            train_df_com = train_df[train_df['Label']==gas]
            test_df_com = test_df[test_df['Label']==gas]
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            groups = train_df_com['Index']
            train_x, train_y, groups = shuffle(train_x, train_y, groups, random_state=42)
            for model_name, model in model_list:
                result, best_param = model_CV(train_x,train_y.squeeze(),groups,model,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                feature_importances = model_refit.feature_importances_.tolist()  #####new lines added to record the feature importance of each round
                test_r2,test_mse = model_eval(model_refit,test_x,test_y.squeeze()) 
                result_total.append([gas,model_name+'_separate',result[0],result[1],-1,-1, test_r2,test_mse,best_param]+feature_importances) # adding feature importance valuels to the end of outputted results
                
                print('Dataset {}, Algorithm {}, Test_r2 {}, Test_error {}'.format(gas,model_name+'_total',test_r2,test_mse))     
    return result_total

In [19]:
BET_df = pd.read_csv("E&E_Adding_bet_80_new.csv")
BET_df.dropna(axis=0,how="any",subset = ["BET","Vt","Vmic","BET_79"],inplace=True)

In [20]:
import os
conditions = ['S','V','Temp(K)','Pressure']
base_feature = ["BET_"+str(i) for i in range(80)]+conditions
feature_list = [base_feature,base_feature+["BET"],base_feature+['Vt'],base_feature+["BET",'Vt'],\
    base_feature+["BET","Vt","Vmic","Vmeso"],["BET","Vt"]+conditions,["BET","Vt","Vmic","Vmeso"]+conditions]
columns = ['Gas','Model_name','CV_r2','CV_mse','test_r2_total_model','test_mse_by_total_model','test_r2_separa_model','test_mse_separa_model','best_param']
file_name = ['BET_Curve',"BET_Curve_BET","BET_Cureve_Vt","BET_Curve_BET_Vt","BET_Curve_BET_Vt_Vmic_Vmeso","BET_Vt","BET_Vt_Vmic_Vmeso"] #['BET_Curve','BET_Plus_Vt',"Vmic_Vmeso"]
gas_list = ['E&E']#,'CFCs','Methane','E&E'b

for i in range(50):
    train_df,test_df = group_split(BET_df,random_state=i)
    for j in range(len(feature_list)):
    
        results = model_comparison(models,para_grids, feature_list[j],gas_list)
        files_name = '80_point_curve_feature_'+file_name[j]+'_result_'+str(i)+'.csv'
        pd.DataFrame(results,columns = columns+feature_list[j]).to_csv(os.path.join('./8_using_BET_curve',files_name)) 
    print("The finish of {} th trial".format(i)) 

Dataset E&E, Algorithm LGBM_total, Test_r2 0.97260839156775, Test_error 0.17775495778473704
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9742126362903083, Test_error 0.16683515628892107
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9671757705347638, Test_error 0.2130604778364036
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9713205424118346, Test_error 0.18773263987392824
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9622120231885529, Test_error 0.24664956426100765
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9516244662605279, Test_error 0.307307220614232
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9594892764484956, Test_error 0.26929382314734174
The finish of 0 th trial
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9488105080923873, Test_error 0.3592902970452879
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9488580313661872, Test_error 0.36225445456857236
Dataset E&E, Algorithm LGBM_total, Test_r2 0.9472392930229372, Test_error 0.3546124085204691
Dataset E&E, Algorithm LGBM_total, Test_r2

# Post result treatments
 

In [22]:
import pandas as pd
file_name = ['BET_Curve',"BET_Curve_BET","BET_Cureve_Vt","BET_Curve_BET_Vt","BET_Curve_BET_Vt_Vmic_Vmeso","BET_Vt","BET_Vt_Vmic_Vmeso"] 
feature_list = [base_feature,base_feature+["BET"],base_feature+['Vt'],base_feature+["BET",'Vt'],\
    base_feature+["BET","Vt","Vmic","Vmeso"],["BET","Vt"]+conditions,["BET","Vt","Vmic","Vmeso"]+conditions]
cal_columns= ["CV_r2","CV_mse","test_r2_separa_model","test_mse_separa_model"]
for j in range(len(file_name)):
    df_list = []
    for i in range(50):
        files_name = '80_point_curve_feature_'+file_name[j]+'_result_'+str(i)+'.csv'

        temp_pd = pd.read_csv(os.path.join('./8_using_BET_curve',files_name))[cal_columns+feature_list[j]]
        temp_pd['feature_sum'] = temp_pd[feature_list[j]].sum(axis=1)
        
        for feature in feature_list[j]:
            temp_pd[feature]=temp_pd[feature]/temp_pd["feature_sum"]
        
        df_list.append(temp_pd)
        pd.concat(df_list).groupby(level=0).mean().to_csv(os.path.join('./8_using_BET_curve',file_name[j]+'_mean.csv'))
        pd.concat(df_list).groupby(level=0).std().to_csv(os.path.join('./8_using_BET_curve',file_name[j]+'_std.csv'))